# title

In [1]:
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [155]:
def get_video_urls(youtube_playlist_url):
    #webdriver setup & html scrape
    driver = webdriver.Chrome('resources/chromedriver')
    driver.get(youtube_playlist_url)
    soup = BeautifulSoup(driver.page_source)

    #isolate the video stats
    video_stats = soup.find_all('yt-formatted-string')[5].text
    #use regex to isolate video number
    no_of_videos = re.sub(',', '', video_stats)
    no_of_videos = re.sub(' videos', '', video_stats)
    #number of scroll based on 100 thumbnails per scroll
    no_of_scrolls = int(int(no_of_videos) / 100 + 1)

    #scroll page for every 100 videos
    for i in range(no_of_scrolls):
        driver.execute_script("window.scrollBy(0, 12000);")
        time.sleep(3)

    soup = BeautifulSoup(driver.page_source)
    driver.quit()

    #extract youtube playlist urls
    matches = soup.find_all('a', class_='yt-simple-endpoint style-scope ytd-playlist-video-renderer')
    youtube_urls = []
    for url in matches:
        string = url.get('href')
        substring = re.search(r'\/watch\?v=\w*', string).group()
        youtube_urls.append('http://www.youtube.com' + substring)    
    return youtube_urls

- title
- url
- account name
- subscribers
- views
- date uploaded
- number of likes
- number of dislikes

In [76]:
def get_stats(youtube_url):
    driver = webdriver.Chrome('resources/chromedriver')
    driver.get(youtube_url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source)
    driver.quit()
    
    def km_cleaner(value):
        if "subscribers" in value:
            if 'K' in value:
                value = float(re.sub('K subscribers', '', value))
                value *= 1000
                return int(value)
            elif 'M' in value:
                value = float(re.sub('M subscribers', '', value))
                value *= 1000000
                return int(value)
            else:
                value = float(re.sub('M subscribers', '', value))
                return int(value)
        else:
            if 'K' in value:
                value = float(re.sub('K', '', value))
                value *= 1000
                return int(value)
            elif 'M' in value:
                value = float(re.sub('M', '', value))
                value *= 1000000
                return int(value)
            else:
                return int(value)
            
    title_value = soup.find_all('yt-formatted-string')[3].text
    
    account_value = soup.find_all('yt-formatted-string')[9].text
    
    subs_string = soup.find_all('yt-formatted-string')[10].text
    subs_value = km_cleaner(subs_string)
    
    view_count_string = soup.find('yt-view-count-renderer').text
    view_count = re.search(r'(\d{1,3},?\d{1,3},?\d{1,3},?\d{1,3},?)', view_count_string).group().replace(',','')
    
    date_value = soup.find_all('yt-formatted-string')[4].text
    
    likes_string = soup.find_all('yt-formatted-string')[5].text
    likes_value = km_cleaner(likes_string)

    dislikes_string = soup.find_all('yt-formatted-string')[6].text
    dislikes_value = km_cleaner(dislikes_string)
    
    stats = {'url': youtube_url,
             'date_uploaded': date_value,
             'views': view_count,
             'account': account_value,
             'video_title': title_value,
             'subscribers': subs_value,
             'likes': likes_value,
             'dislikes': dislikes_value
            }
    
    return stats 

In [156]:
playlist = get_video_urls('https://www.youtube.com/playlist?list=PL7qtZGedQPacVMB47TSAROx1VgTQdV96N')

In [139]:
driver = webdriver.Chrome('resources/chromedriver')
driver.get('https://www.youtube.com/playlist?list=PLWwAypAcFRgKAIIFqBr9oy-ZYZnixa_Fj')
soup = BeautifulSoup(driver.page_source)
driver.quit()

In [146]:
hit = soup.find_all('yt-formatted-string')[5]

In [147]:
hit.text

'5,000 videos'

In [131]:
for index, value in enumerate(hit):
    print(index, value)

0 <yt-formatted-string class="style-scope ytd-button-renderer style-default size-default" id="text">Skip navigation</yt-formatted-string>
1 <yt-formatted-string class="style-scope ytd-button-renderer style-suggestive size-small" id="text">Sign in</yt-formatted-string>
2 <yt-formatted-string class="style-scope ytd-playlist-thumbnail" hidden="" id="length"><!--css-build:shady--></yt-formatted-string>
3 <yt-formatted-string class="style-scope ytd-thumbnail-overlay-side-panel-renderer">PLAY ALL</yt-formatted-string>
4 <yt-formatted-string class="style-scope ytd-playlist-sidebar-primary-info-renderer" has-link-only_=""><a class="yt-simple-endpoint style-scope yt-formatted-string" dir="auto" href="/watch?v=8Kp8WKodb3M&amp;list=PL7qtZGedQPacVMB47TSAROx1VgTQdV96N" spellcheck="false">Mix</a></yt-formatted-string>
5 <yt-formatted-string class="style-scope ytd-playlist-sidebar-primary-info-renderer">4 videos</yt-formatted-string>
6 <yt-formatted-string class="style-scope ytd-playlist-sidebar-prim

In [108]:
test = int(int('4') / 100) + 1

In [109]:
test

1

In [113]:
int(int('4') / 100) + 1

1

In [127]:
test = re.search('(\d{0,3},?\d{0,3},?)', gerard).group()

In [128]:
test

''

In [124]:
gerard

'\n4 videos\nUpdated yesterday\n'

In [129]:
gerard

'\n4 videos\nUpdated yesterday\n'